# Previsão de notas dos alunos
Muitos alunos de pós-graduação têm dificuldade em obter boas notas porque não recebem muito apoio nos cursos superiores em comparação com o apoio que os alunos recebem nas escolas. Alguns alunos precisam de muita atenção dos instrutores para que obtenham boas notas, sem isso, o estado emocional do aluno pode ser prejudicial para a sua carreira a longo prazo.

**O objetivo desse projeto é, através do aprendizado de máquina, prever as notas dos alunos para que os instrutores possam ajudar os alunos a se prepararem para tópicos em que as notas dos alunos foram previstas baixas.**  



https://www.kaggle.com/code/ramontanoeiro/student-performance/notebook

# #2 - Tratamento dos dados
|                                       | Numéricos serão normalizados                                         | Numéricos não serão normalizados                                         |
|---------------------------------------|----------------------------------------------------------------------|--------------------------------------------------------------------------|
| **Ignorar multicolinearidade**            | Ignorar multicolinearidade e Numéricos serão normalizados            | Ignorar multicolinearidade e Numéricos não serão normalizados            |
| **Remover multicolinearidade alta**       | Remover multicolinearidade alta e Numéricos serão normalizados       | Remover multicolinearidade alta e Numéricos não serão normalizados       |
| **Remover multicolinearidade muito alta** | Remover multicolinearidade muito alta e Numéricos serão normalizados | Remover multicolinearidade muito alta e Numéricos não serão normalizados |

https://www.analyticsvidhya.com/blog/2016/07/deeper-regression-analysis-assumptions-plots-solutions/?utm_source=blog&utm_medium=one-hot-encoding-vs-label-encoding-using-scikit-learn

Algoritmos que utilizarão a base de dados com numéricos **não** normalizados
- from sklearn.tree import DecisionTreeRegressor
- from sklearn.ensemble import RandomForestRegressor
- from gboost import XGBRegressor

Algoritmos que utilizarão a base de dados com numéricos normalizados
- from sklearn.svm import SVR
- from sklearn.neural_network import MLPRegressor
- from sklearn.neighbors import KNeighborsRegressor


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose  import ColumnTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pandas as pd
import numpy as np
import math
import pickle as pkl
path_datasets = '/content/drive/MyDrive/Machine Learning e Data Science com Python/Projeto - Nota dos Alunos/'

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 150)
df = pd.read_csv(path_datasets+'student-mat.csv')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,course,other,1,1,3,no,no,no,no,no,yes,no,no,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother,3,1,0,no,no,no,no,no,yes,yes,no,4,4,1,3,4,5,0,11,12,10


## Separando features do target

In [4]:
X, y = df.iloc[:, 0:32], df.iloc[:, 32]
X.shape, y.shape

((395, 32), (395,))

In [5]:
features_num = list(X.select_dtypes(include=['int64']).columns)
features_cat = list(X.select_dtypes(include=['object']).columns)

## Removendo colunas com multicolinearidade
O problema da multicolinearidade é que ela afeta a variância das estimativas dos coeficientes e as torna sensíveis a pequenas mudanças no modelo. Em resumo, ela dificulta a especificação do modelo correto. @universidadedosdados  

A análise de multicolinearidade está na sessão #1 - Exploração dos dados

In [6]:
def remove_multic_alta(df):
  cols_remove = ['age', 'famrel', 'G1', 'G2', 'Medu', 'Fedu', 'freetime', 'goout']
  features_num_MA = [col for col in features_num if col not in cols_remove]
  return df.drop(columns=cols_remove, axis = 1), features_num_MA

def remove_multic_muito_alta(df):
  cols_remove = ['age', 'famrel', 'G1', 'G2']
  features_num_MMA = [col for col in features_num if col not in cols_remove]
  return df.drop(columns=cols_remove, axis = 1), features_num_MMA

In [7]:
X_MA, features_num_MA   = remove_multic_alta(X)
X_MMA, features_num_MMA = remove_multic_muito_alta(X)

X_MA.shape, X_MMA.shape

((395, 24), (395, 28))

## Transformando categóricos com One Hot Encoder

In [8]:
Cat_transformer = Pipeline(steps=[('one-hot encoder', OneHotEncoder())])

## Normalizando numéricos com MinMaxScaler
Dado que os dados não estão em uma distribuição normal e, como também, não desejamos eliminar a influência dos outliers, então iremos utilizar o MinMaxScaler

In [9]:
Num_transformer = Pipeline(steps = [('min-max-scaler',  MinMaxScaler())])

## Compondo os pré-processadores

In [10]:
# Base de dados completa & Normalizada
Preprocessor = ColumnTransformer(transformers=[
    ('num', Num_transformer, features_num),
    ('cat', Cat_transformer, features_cat)
])

In [11]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
Preprocessor_MMA = ColumnTransformer(transformers=[
    ('num', Num_transformer, features_num_MMA),
    ('cat', Cat_transformer, features_cat)
])

In [12]:
# Base de dados sem features com multicolinearidade alta & Normalizada
Preprocessor_MA = ColumnTransformer(transformers=[
    ('num', Num_transformer, features_num_MA),
    ('cat', Cat_transformer, features_cat)
])

In [13]:
# Base de dados completa & Não Normalizada
Preprocessor_NN = ColumnTransformer(transformers=[
    ('cat', Cat_transformer, features_cat)
])

In [14]:
# Base de dados sem features com multicolinearidade muito alta & Não Normalizada
Preprocessor_MMA_NN = ColumnTransformer(transformers=[
    ('cat', Cat_transformer, features_cat)
])

In [15]:
# Base de dados sem features com multicolinearidade alta & Não Normalizada
Preprocessor_MA_NN = ColumnTransformer(transformers=[
    ('cat', Cat_transformer, features_cat)
])

## Base de treino e teste

In [16]:
seed = 14

In [17]:
# Base de dados completa & Normalizada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = seed)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((296, 32), (296,), (99, 32), (99,))

In [18]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
X_MMA_train, X_MMA_test, y_train, y_test = train_test_split(X_MMA, y, test_size = .25, random_state = seed)
X_MMA_train.shape, y_train.shape, X_MMA_test.shape, y_test.shape

((296, 28), (296,), (99, 28), (99,))

In [19]:
# Base de dados sem features com multicolinearidade alta & Normalizada
X_MA_train, X_MA_test, y_train, y_test = train_test_split(X_MA, y, test_size = .25, random_state = seed)
X_MA_train.shape, y_train.shape, X_MA_test.shape, y_test.shape

((296, 24), (296,), (99, 24), (99,))

In [20]:
# Base de dados completa & Não Normalizada
X_NN_train, X_NN_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = seed)
X_NN_train.shape, y_train.shape, X_NN_test.shape, y_test.shape

((296, 32), (296,), (99, 32), (99,))

In [21]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
X_MMA_NN_train, X_MMA_NN_test, y_train, y_test = train_test_split(X_MMA, y, test_size = .25, random_state = seed)
X_MMA_NN_train.shape, y_train.shape, X_MMA_NN_test.shape, y_test.shape

((296, 28), (296,), (99, 28), (99,))

In [22]:
# Base de dados sem features com multicolinearidade alta & Não Normalizada
X_MA_NN_train, X_MA_NN_test, y_train, y_test = train_test_split(X_MA, y, test_size = .25, random_state = seed)
X_MA_NN_train.shape, y_train.shape, X_MA_NN_test.shape, y_test.shape

((296, 24), (296,), (99, 24), (99,))

In [23]:
OneHotColumns = ['school_GP', 'school_MS',
                'sex_F', 'sex_M', 'address_R',
                'address_U', 'famsize_GT3', 'famsize_LE3',
                'Pstatus_A', 'Pstatus_T', 'Mjob_at_home',
                'Mjob_health', 'Mjob_other',
                'Mjob_services', 'Mjob_teacher', 'Fjob_at_home',
                'Fjob_health','Fjob_other', 'Fjob_services',
                'Fjob_teacher', 'reason_course','reason_home',
                'reason_other', 'reason_reputation',
                'guardian_father', 'guardian_mother',
                'guardian_other', 'schoolsup_no', 'schoolsup_yes',
                'famsup_no', 'famsup_yes', 'paid_no', 'paid_yes',
                'activities_no', 'activities_yes', 'nursery_no',
                'nursery_yes', 'higher_no', 'higher_yes',
                'internet_no', 'internet_yes', 'romantic_no',
                'romantic_yes']

In [24]:
columns_X     = features_num + OneHotColumns
columns_X_MA  = features_num_MA + OneHotColumns
columns_X_MMA = features_num_MMA + OneHotColumns

### Treinando e transformando base de treino

In [25]:
# Base de dados completa & Normalizada
X_train_transformed = Preprocessor.fit_transform(X_train, y_train)
X_train_transformed = pd.DataFrame(X_train_transformed, columns = columns_X)
X_train_transformed.shape

(296, 58)

In [26]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
X_MMA_train_transformed = Preprocessor_MMA.fit_transform(X_MMA_train, y_train)
X_MMA_train_transformed = pd.DataFrame(X_MMA_train_transformed, columns = columns_X_MMA)
X_MMA_train_transformed.shape

(296, 54)

In [27]:
# Base de dados sem features com multicolinearidade alta & Normalizada
X_MA_train_transformed = Preprocessor_MA.fit_transform(X_MA_train, y_train)
X_MA_train_transformed = pd.DataFrame(X_MA_train_transformed, columns = columns_X_MA)
X_MA_train_transformed.shape

(296, 50)

In [28]:
# Base de dados completa & Não Normalizada
X_NN_train_transformed = Preprocessor_NN.fit_transform(X_NN_train, y_train)
X_NN_train_transformed = pd.DataFrame(X_NN_train_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_NN_train_transformed = pd.concat([X_NN_train[features_num].reset_index(drop=True), X_NN_train_transformed], axis = 1)
X_NN_train_transformed.shape

(296, 58)

In [29]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
X_MMA_NN_train_transformed = Preprocessor_MMA_NN.fit_transform(X_MMA_NN_train, y_train)
X_MMA_NN_train_transformed = pd.DataFrame(X_MMA_NN_train_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_MMA_NN_train_transformed = pd.concat([X_MMA_NN_train[features_num_MMA].reset_index(drop=True), X_MMA_NN_train_transformed], axis = 1)
X_MMA_NN_train_transformed.shape

(296, 54)

In [30]:
# Base de dados sem features com multicolinearidade alta & Não Normalizada
X_MA_NN_train_transformed = Preprocessor_MA_NN.fit_transform(X_MA_NN_train, y_train)
X_MA_NN_train_transformed = pd.DataFrame(X_MA_NN_train_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_MA_NN_train_transformed = pd.concat([X_MA_NN_train[features_num_MA].reset_index(drop=True), X_MA_NN_train_transformed], axis = 1)
X_MA_NN_train_transformed.shape

(296, 50)

### Treinando e transformando target

In [31]:
y_scaler = MinMaxScaler()
y_train_transformed = y_scaler.fit_transform(y_train.array.reshape(-1, 1))
y_train_transformed[0:5]

array([[0.5 ],
       [0.85],
       [0.5 ],
       [0.75],
       [0.9 ]])

### Tratando Dummy Variable Trap
A armadilha Dummy Variable é um cenário em que as variáveis ​​independentes são multicolineares - um cenário em que duas ou mais variáveis ​​são altamente correlacionadas; em termos simples, uma variável pode ser prevista a partir das outras.  

Assim, para superar o problema da multicolinearidade, uma das variáveis ​​dummy deve ser eliminada.  

Uma das maneiras comuns de verificar a multicolinearidade é o fator de inflação de variância (VIF):  

- VIF=1, muito menos multicolinearidade
- VIF=5, Multicolinearidade Moderada
- VIF>5, Multicolinearidade Extrema (Isto é o que temos que evitar)

**fontes:**  
*https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/*

*https://www.algosome.com/articles/dummy-variable-trap-regression.html*

Vamos demonstrar o VIF da base de dados original transformada. As variáveis independentes dummies possuem multicolinearidade perfeita, o que prova que temos que remover algumas dessas variáveis.

In [32]:
X_columns = X_train_transformed.columns

In [33]:
vif = pd.DataFrame()
vif['feature'] = X_columns
vif["VIF"] = [variance_inflation_factor(X_train_transformed[X_columns].values, i) for i in range(len(X_columns))]
vif

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
0,age,1.974428
1,Medu,3.089285
2,Fedu,2.097746
3,traveltime,1.366733
4,studytime,1.467127
5,failures,1.654673
6,famrel,1.165120
7,freetime,1.408093
8,goout,1.572792
9,Dalc,2.050947


In [34]:
OneHotColumns_include = []
OneHotColumns_exclude = []

for c in OneHotColumns:
  try:
    c1, c2 = c.split('_')
  except:
    c1, c2, c3 = c.split('_')
    c2 = f'{c2}_{c3}'

  if 'other' in c2:
    OneHotColumns_exclude.append(c)
  elif c1 not in OneHotColumns_include:
    OneHotColumns_include.append(c1)
  else:
    OneHotColumns_exclude.append(c)
OneHotColumns_exclude

['school_MS',
 'sex_M',
 'address_U',
 'famsize_LE3',
 'Pstatus_T',
 'Mjob_health',
 'Mjob_other',
 'Mjob_services',
 'Mjob_teacher',
 'Fjob_health',
 'Fjob_other',
 'Fjob_services',
 'Fjob_teacher',
 'reason_home',
 'reason_other',
 'reason_reputation',
 'guardian_mother',
 'guardian_other',
 'schoolsup_yes',
 'famsup_yes',
 'paid_yes',
 'activities_yes',
 'nursery_yes',
 'higher_yes',
 'internet_yes',
 'romantic_yes']

In [35]:
def drop_columns(df, columns_list):
  return df.drop(columns=columns_list, axis = 1)

In [36]:
X_train_transformed        = drop_columns(X_train_transformed, OneHotColumns_exclude)
X_MA_train_transformed     = drop_columns(X_MA_train_transformed, OneHotColumns_exclude)
X_MMA_train_transformed    = drop_columns(X_MMA_train_transformed, OneHotColumns_exclude)
X_NN_train_transformed     = drop_columns(X_NN_train_transformed, OneHotColumns_exclude)
X_MA_NN_train_transformed  = drop_columns(X_MA_NN_train_transformed, OneHotColumns_exclude)
X_MMA_NN_train_transformed = drop_columns(X_MMA_NN_train_transformed, OneHotColumns_exclude)

### Transformando base de teste

In [37]:
# Base de dados completa & Normalizada
X_test_transformed = Preprocessor.transform(X_test)
X_test_transformed = pd.DataFrame(X_test_transformed, columns = columns_X)
X_test_transformed.shape

(99, 58)

In [38]:
# Base de dados sem features com multicolinearidade muito alta & Normalizada
X_MMA_test_transformed = Preprocessor_MMA.transform(X_MMA_test)
X_MMA_test_transformed = pd.DataFrame(X_MMA_test_transformed, columns = columns_X_MMA)
X_MMA_test_transformed.shape

(99, 54)

In [39]:
# Base de dados sem features com multicolinearidade alta & Normalizada
X_MA_test_transformed = Preprocessor_MA.transform(X_MA_test)
X_MA_test_transformed = pd.DataFrame(X_MA_test_transformed, columns = columns_X_MA)
X_MA_test_transformed.shape

(99, 50)

In [40]:
# Base de dados completa & Não Normalizada
X_NN_test_transformed = Preprocessor_NN.transform(X_NN_test)
X_NN_test_transformed = pd.DataFrame(X_NN_test_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_NN_test_transformed = pd.concat([X_NN_test[features_num].reset_index(drop=True), X_NN_test_transformed], axis = 1)
X_NN_test_transformed.shape

(99, 58)

In [41]:
# Base de dados sem features com multicolinearidade muito alta & Não Normalizada
X_MMA_NN_test_transformed = Preprocessor_MMA_NN.transform(X_MMA_NN_test)
X_MMA_NN_test_transformed = pd.DataFrame(X_MMA_NN_test_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_MMA_NN_test_transformed = pd.concat([X_MMA_NN_test[features_num_MMA].reset_index(drop=True), X_MMA_NN_test_transformed], axis = 1)
X_MMA_NN_test_transformed.shape

(99, 54)

In [42]:
# Base de dados sem features com multicolinearidade alta & Não Normalizada
X_MA_NN_test_transformed = Preprocessor_MA_NN.transform(X_MA_NN_test)
X_MA_NN_test_transformed = pd.DataFrame(X_MA_NN_test_transformed, columns = OneHotColumns)
  ## Concatenando features numéricas sem normalização com categóricas tratadas
X_MA_NN_test_transformed = pd.concat([X_MA_NN_test[features_num_MA].reset_index(drop=True), X_MA_NN_test_transformed], axis = 1)
X_MA_NN_test_transformed.shape

(99, 50)

### Tratando Dummy Variable Trap pt2

In [43]:
X_test_transformed        = drop_columns(X_test_transformed, OneHotColumns_exclude)
X_MA_test_transformed     = drop_columns(X_MA_test_transformed, OneHotColumns_exclude)
X_MMA_test_transformed    = drop_columns(X_MMA_test_transformed, OneHotColumns_exclude)
X_NN_test_transformed     = drop_columns(X_NN_test_transformed, OneHotColumns_exclude)
X_MA_NN_test_transformed  = drop_columns(X_MA_NN_test_transformed, OneHotColumns_exclude)
X_MMA_NN_test_transformed = drop_columns(X_MMA_NN_test_transformed, OneHotColumns_exclude)

### Transformando target

In [44]:
y_test_transformed = y_scaler.transform(y_test.array.reshape(-1, 1))
y_test_transformed[0:5]

array([[0.  ],
       [0.35],
       [0.45],
       [0.3 ],
       [0.6 ]])

## Salvando as variáveis

In [45]:
data_list = [X_train_transformed, X_MA_train_transformed, X_MMA_train_transformed,
             X_test_transformed , X_MA_test_transformed , X_MMA_test_transformed ,
             X_NN_train_transformed, X_MA_NN_train_transformed, X_MMA_NN_train_transformed,
             X_NN_test_transformed , X_MA_NN_test_transformed , X_MMA_NN_test_transformed,
             y_train_transformed, y_test_transformed]

In [46]:
transformers_list = [Preprocessor, Preprocessor_MA, Preprocessor_MMA,
                     Preprocessor_NN, Preprocessor_MA_NN, Preprocessor_MMA_NN,
                     y_scaler]

In [47]:
with open(path_datasets+'students_data.pkl', 'wb') as f:
  pkl.dump(data_list, f)
print('OK')

OK


In [48]:
with open(path_datasets+'students_preprocessor.pkl', 'wb') as f:
  pkl.dump(transformers_list, f)
print('OK')

OK
